In [1]:
# Model-1-V4
# - Load data set
# - Remove NaN in short_desc
# - Convert to lowercase
# - Split the words using 1. ASCII character identification for english 2. Split by Space  3. wordninja
# - Apply normlise
# - Apply contractions/expansions 
# - Remove punctuations
# - remove tags.
# - remove special characters and digits.
# - Stemming/Lemmatisation. 
# - Prepare the whole data set
# - Calculate the similarity between every two bugreport for the same product

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
data_org = pd.read_csv('https://zenodo.org/record/400614/files/apache.csv?download=1',sep=',')
# data.head()

In [4]:
#Problem in reading csv file, cut the short_desc after 88th character
# print(data.iloc[[124]]['short_desc'])

In [5]:
data = data_org[['id','product','short_desc']]
# data.head()

In [6]:
def remove_nan(df):
    blanks = []
    print("Before removing the NaN:")
    print(df.isnull().sum())
    for x in df.itertuples():
        if type(x.short_desc)!=str: # detect the NaN
            blanks.append(x.Index)
        elif not x.short_desc: # detect empty string
            blanks.append(x.Index)
    df.drop(blanks,inplace=True)
    print("\nAfter removing the NaN:\n",df.isnull().sum())

In [7]:
remove_nan(data)

Before removing the NaN:
id             0
product        0
short_desc    58
dtype: int64

After removing the NaN:
 id            0
product       0
short_desc    0
dtype: int64


In [8]:
# conda install -c conda-forge spacy
# !pip install wordninja
# !pip install normalise
# !pip install pycontractions
# conda install -c conda-forge spacy-lookups-data
# !python -m spacy download en_core_web_lg

In [9]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_lg')
import nltk
# nltk.download('brown')
# nltk.download('names')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('universal_tagset')
import wordninja as nj #for spliting the words in each documents
import normalise as ns 
from pycontractions import Contractions # for expansion and contrations
import contractions
import re # remove tags.

def preprocessing(df,id):
    # Extract and convert short_desc to string
    row = df[df['id'] == int(id)]                                # Extract a tuple from the dataframe
    short_desc_to_string = row.short_desc.to_string(index=False) # Conver short_desc to string
    short_desc_to_string = short_desc_to_string[1:]              # Remove the first space char from the begnning

#     my_short_desc_to_string = "I'd liked to said <pclass='test'> CANADAisn't Alireza@gmail.com the worst-countries in the 44 world!!!"
#     print("Original:\n" + my_short_desc_to_string)
    
    # Convert to lowercase
    short_desc_lowercase = short_desc_to_string.lower()
#     print("\nshort_desc_lowercase:\n" + short_desc_lowercase)
    
    # Split the words using 1. ASCII character identification for english 2. Split by Space  3. wordninja
    short_desc_splited = nj.split(short_desc_lowercase)
#     print("\nshort_desc_splited:\n" + str(short_desc_splited))
    
    # Apply normlise
    short_desc_normalised = ns.normalise(short_desc_splited,verbose=False)
    short_desc_normalised_listToStr = ' '.join(map(str, short_desc_normalised))
    
#     print("\nshort_desc_normalised:\n" + short_desc_normalised_listToStr)
    
    # Apply contractions/expansions 
#     cont = Contractions(api_key="glove-twitter-100") #We shold use this one but it shows error 
    short_desc_contract = contractions.fix(short_desc_normalised_listToStr) 
#     print("\nshort_desc_contract:\n" + str(short_desc_contract))
        
    # remove tags.
    clean = re.compile('<.*?>')
    short_desc_removed_tag = re.sub(clean, '', str(short_desc_contract)) 
#     print("\nshort_desc_removed_tag:\n" + str(short_desc_removed_tag))
    
    # remove special characters and digits.
    short_desc_removed_special_char = [re.sub(r"[^a-zA-Z]+", ' ', k) for k in str(short_desc_removed_tag).split("\n")]
#     print("\nshort_desc_removed_special_char:\n" + str(short_desc_removed_special_char))

    # Remove Punctuations and Stop words
    short_desc_doc = nlp(str(short_desc_removed_special_char))  
    short_desc_map = map(lambda token: token if (token.is_punct == False and token.is_stop == False) else None, short_desc_doc)
    short_desc_list = list(short_desc_map)        
    
    # Convert list to string and remove one-character word
    short_desc_string = ""     
    for element in short_desc_list:
        if element != None and len(element)>1:
            short_desc_string += str(element) +' '
    short_desc_string = short_desc_string[:-1]
#     print("\nshort_desc_string-1:\n" + short_desc_string)

#     short_desc_string2 = ' '.join(map(lambda x: str(x) if (x != None and len(x)>1) else None, short_desc_list))
#     print("\nshort_desc_string-2:\n" + short_desc_string2)
            
    # Stemming/Lemmatisation.       
    short_desc_lemmata = [token.lemma_ for token in nlp(short_desc_string)]
#     print("short_desc_lemmata:\n" + str(short_desc_lemmata))
    short_desc_preprocessed = ' '.join(map(str, short_desc_lemmata))
        
    return short_desc_preprocessed

In [79]:
# Prepare the vectors for each document
import time
import progressbar as pb
start_time = time.time()

sample_size = 5 # Sample Size
progress = pb.ProgressBar(maxval = sample_size).start()

# dataset_length = len(data)
# progress = pb.ProgressBar(maxval = dataset_length).start()

progvar = 0
processed_data_list = []

for x in data.itertuples():
    string = preprocessing(data,x.id)
    processed_data_list.append((x.id,x.product,string))   
    
    # Show the progress in the output
    progress.update(progvar+1)  
    progvar += 1    
    
    # Terminate the process when reach to sample size
    if progvar >= sample_size:
        break
# Convert list to dataframe
processed_data_df = pd.DataFrame(processed_data_list, columns=['id','product','short_desc'])
# del data
del processed_data_list
# show the time of process        
print("--- %s seconds ---" % (time.time() - start_time))
processed_data_df.head()

 80% (4 of 5) |####################      | Elapsed Time: 0:00:00 ETA:   0:00:00

--- 0.3636445999145508 seconds ---


,id,product,short_desc
0,2,Log4j -,testing book zee la setup log
1,3,Log4j -,file share
2,22,Apache h,need hook user define error handle
3,27,Log4j -,error forget layout
4,29,Log4j -,call category error object throw able thro


In [80]:
processed_data_df.to_csv('~/Desktop/Google-Drive/Colab Notebooks/processed_data_df.csv',index=False)

In [ ]:
def index_to_id(df):
    length = len(df)
    for tup in df.itertuples():
        id1 = data_org.iloc[tup.index1]['id']
        id2 = data_org.iloc[tup.index2]['id']
        df.loc[tup.Index,'index1'] = id1
        df.loc[tup.Index,'index2'] = id2

Model-1: Similarity Score - Word2vec

In [108]:
# Calculate similarity score Using word2vec
start_time = time.time()
dataset_length = len(processed_data_df)
progress = pb.ProgressBar(maxval = dataset_length).start()
progvar = 0

# Convert short_desc str to nlp format to fasten the computation process
processed_data_nlp = []

for tup in processed_data_df.itertuples():
    processed_data_nlp.append((tup.id,tup.product,nlp(tup.short_desc)))
    progress.update(progvar+1)  
    progvar += 1
    
processed_data_nlp_df = pd.DataFrame(processed_data_nlp, columns=['id','product','short_desc'])
processed_data_nlp_df.head()
del processed_data_nlp

# Calculate the similarity score
progress = pb.ProgressBar(maxval = dataset_length).start()

similarities_score_list = []

for doc1 in processed_data_nlp_df.itertuples():
    for doc2 in processed_data_nlp_df.itertuples():
        if (doc1.id != doc2.id) and (doc1.product == doc2.product): # if two bug reports belong to the same product then check the similarity
            similarity_score = doc1.short_desc.similarity(doc2.short_desc)
            similarities_score_list.append((doc1.id,doc2.id,similarity_score))         
    # Show the progress in the output        
    progress.update(doc1.Index+1)  
    

# Sort based on the score and 
sorted_similarities_score = sorted(similarities_score_list, key=lambda tup: tup[2], reverse=True)
del similarities_score_list

#convert to dataframe
word2vec_similarities_score_df = pd.DataFrame(sorted_similarities_score, columns=['index1','index2','score'] )
del sorted_similarities_score

print("--- %s seconds ---" % (time.time() - start_time)) # show the time of process
word2vec_similarities_score_df.head()

N/A% (0 of 5) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--

--- 0.09471583366394043 seconds ---


,index1,index2,score
0,27,29,0.728128
1,29,27,0.728128
2,2,27,0.582265
3,27,2,0.582265
4,3,27,0.551018


In [109]:
# processed_data_df.to_csv('~/Desktop/Google-Drive/Colab Notebooks/word2vec_similarities_score_df.csv',index=False)

Model-2: Similarity Score - tf-idf

In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

X_train = processed_data_df['short_desc']

start_time = time.time()

# Vectorization 
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set

# Similarity score calculation
tfidf_cosine_similarities = linear_kernel(tfidf)

shape = tfidf_cosine_similarities.shape
progress = pb.ProgressBar(maxval = shape[0]).start()

tfidf_cosine_similarities_list = []

for index1 in range(shape[0]):
    for index2 in range(shape[1]):
        if index2 > index1:
            tfidf_cosine_similarities_list.append([index1,index2,tfidf_cosine_similarities[index1,index2]])
            
    progress.update(index1 + 1)  
    
# Sort the score list
tfidf_cosine_similarities_sort = sorted(tfidf_cosine_similarities_list, key=lambda tup: tup[2], reverse=True)
del tfidf_cosine_similarities_list

#Conver to dataframe
tfidf_cosine_similarities_score_df = pd.DataFrame(tfidf_cosine_similarities_sort, columns=['index1','index2','score'])
del tfidf_cosine_similarities_sort

# Convert index to id
index_to_id(tfidf_cosine_similarities_score_df)
tfidf_cosine_similarities_score_df.rename(columns={"index1": "id1", "index2": "id2"})

print("--- %s seconds ---" % (time.time() - start_time)) # show the time of process
tfidf_cosine_similarities_score_df.head()

N/A% (0 of 5) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--

--- 0.0071184635162353516 seconds ---


,index1,index2,score
0,2,3,0.122796
1,3,4,0.112874
2,2,4,0.075667
3,0,1,0.000000
4,0,2,0.000000


In [111]:
# processed_data_df.to_csv('~/Desktop/Google-Drive/Colab Notebooks/tfidf_cosine_similarities_df.csv',index=False)

Model-3: Similarity Score - BM24F

In [112]:
from rank_bm25 import BM25Okapi

start_time = time.time()
progress = pb.ProgressBar(maxval = len(processed_data_df)).start()

# preprocess - tokenize the short_desc to token
processed_corpus_list = []
for x in processed_data_df.itertuples():
    short_desc_splited = x.short_desc.split(" ")
    processed_corpus_list.append(short_desc_splited)
    
# Create a MB24 Object with the corpus
bm25 = BM25Okapi(processed_corpus_list)

# Calculat the similarity score for all bug reports
BM24_similarity_score_list = []

for x in processed_data_df.itertuples():
    query = processed_corpus_list[x.Index]
    doc_scores = bm25.get_scores(query)
    BM24_similarity_score_list.append([x.Index,doc_scores])

# Reformating 
shape = doc_scores.shape[0]
BM24_similarity_score_list_2 = []

for index1 in range(shape):
    for index2 in range(shape):
        if index2 > index1:
            BM24_similarity_score_list_2.append([index1+2,index2+2,BM24_similarity_score_list[index1][1][index2]])  
    
    progress.update(index1 + 1)  
            
del BM24_similarity_score_list

# Sort the score list
BM24_similarity_score_sort = sorted(BM24_similarity_score_list_2, key=lambda tup: tup[2], reverse=True)
del BM24_similarity_score_list_2

#Conver to dataframe
BM24_similarity_score_sort_df = pd.DataFrame(BM24_similarity_score_sort, columns=['index1','index2','score'])
del BM24_similarity_score_sort

print("--- %s seconds ---" % (time.time() - start_time)) # show the time of process

# Convert index to id
index_to_id(BM24_similarity_score_sort_df)
BM24_similarity_score_sort_df.rename(columns={"index1": "id1", "index2": "id2"})

BM24_similarity_score_sort_df.head()
    
# bm25.get_top_n(tokenized_query, processed_corpus_list, n=1)

N/A% (0 of 5) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--

--- 0.007170915603637695 seconds ---


,index1,index2,score
0,4,5,0.310791
1,4,6,0.214172
2,5,6,0.214172
3,2,3,0.000000
4,2,4,0.000000


Assessment the Model Accuracy

In [113]:
duplicat_org = pd.read_csv('https://zenodo.org/record/400614/files/apache.relations.csv?download=1',sep=',')